In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
from utils.classUtils.Analysis import dependency, required, MethodDependency

In [4]:
class test_analysis(Analysis):

    def func_a(self):
        self.a = 0

    @required
    def func_b(self):
        self.b = self.a + 10

    @dependency(func_a)
    def plot_a(self):
        print('a =',self.a+0)

    @dependency(func_a, func_b)
    def calc_c(self):
        self.c = self.a + self.b 

    @dependency(calc_c)
    def plot_c(self):
        print(self.c)


In [5]:
MethodDependency._dependency_graphs

defaultdict(<function utils.classUtils.Analysis.MethodDependency.<lambda>()>,
            {'eightb_analysis_v2': defaultdict(list,
                         {'_plot_unassigned_higgs_m': ['build_higgs_dm'],
                          'plot_assigned_higgs_m': ['build_higgs_dm'],
                          'plot_y_higgs_m': ['build_higgs_dm'],
                          'plot_abcd_variables': ['build_higgs_dm'],
                          'build_abcd': ['build_higgs_dm'],
                          'plot_abcd_regions': ['build_abcd'],
                          'plot_vr_datamc': ['build_abcd'],
                          'train_vr_bdt': ['build_abcd'],
                          'train_ar_bdt': ['build_abcd'],
                          'calc_limits': ['train_ar_bdt'],
                          'calc_brazil': ['train_ar_bdt']}),
             'test_analysis': defaultdict(list,
                         {'plot_a': ['func_a'],
                          'calc_c': ['func_a', 'func_b'],
                  

In [6]:
runlist = ['plot_c']

In [8]:
analysis = test_analysis(runlist=runlist)
analysis

Analysis: test_analysis
<MethodList
    0: <[pending ] func_a>
    1: <[pending ] func_b>
    2: <[disabled] plot_a>
    3: <[pending ] calc_c>
    4: <[pending ] plot_c>
>

In [9]:
analysis.dependency.dependency_graph

defaultdict(list, {})

In [7]:
method_list = analysis.methods.keys
required_graph = MethodDependency.get_required(test_analysis)
dependency_graph = MethodDependency.get_dependency(test_analysis)

In [8]:

def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [12]:
for required in required_graph:
    idx = method_list.index(required)
    for method in method_list[idx+1:]:
        dependency_graph[method] = f7(dependency_graph[method] + [required])

for method, dependency in dependency_graph.items():
    dependency_graph[method] = sorted(dependency, key=method_list.index)

In [13]:
dependency_graph

defaultdict(list,
            {'plot_a': ['func_a', 'func_b'],
             'calc_c': ['func_a', 'func_b'],
             'plot_c': ['func_b', 'calc_c']})

In [14]:
analysis.run(runlist)

AttributeError: 'function' object has no attribute 'build_runlist'